In [ ]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar -xf imagenette2.tgz

--2022-12-10 23:43:37--  https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.142.168, 52.216.108.5, 52.217.195.32, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.142.168|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1557161267 (1.5G) [application/x-tar]
Saving to: ‘imagenette2.tgz’

imagenette2.tgz     100%[===================>]   1.45G  57.5MB/s    in 28s     

2022-12-10 23:44:05 (53.6 MB/s) - ‘imagenette2.tgz’ saved [1557161267/1557161267]



In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout
import matplotlib.pyplot as plt

In [ ]:
imagenette_map = { 
    "n01440764" : "tench",
    "n02102040" : "springer",
    "n02979186" : "casette_player",
    "n03000684" : "chain_saw",
    "n03028079" : "church",
    "n03394916" : "French_horn",
    "n03417042" : "garbage_truck",
    "n03425413" : "gas_pump",
    "n03445777" : "golf_ball",
    "n03888257" : "parachute"
}

In [ ]:
# create a new generator
imagegen = ImageDataGenerator()
# load train data
train = imagegen.flow_from_directory("imagenette2/train/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))
# load val data
val = imagegen.flow_from_directory("imagenette2/val/", class_mode="categorical", shuffle=False, batch_size=128, target_size=(224, 224))

Found 9469 images belonging to 10 classes.
Found 3925 images belonging to 10 classes.


In [ ]:
# build a sequential model
model = Sequential()
model.add(InputLayer(input_shape=(224, 224, 3)))

# 1st conv block
model.add(Conv2D(25, (5, 5), activation='relu', strides=(1, 1), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
# 2nd conv block
model.add(Conv2D(50, (5, 5), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='same'))
model.add(BatchNormalization())
# 3rd conv block
model.add(Conv2D(70, (3, 3), activation='relu', strides=(2, 2), padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), padding='valid'))
model.add(BatchNormalization())
# ANN block
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.25))
# output layer
model.add(Dense(units=10, activation='softmax'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 25)      1900      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 25)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 56, 56, 50)        31300     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 50)       0         
 2D)                                                             
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 50)       200       
 ormalization)                                                   
                                                        

In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
# fit on data for 30 epochs
model1 = model.fit_generator(train, epochs=10, validation_data=val)

<ipython-input-7-05c0d4711c40>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model1 = model.fit_generator(train, epochs=10, validation_data=val)


Epoch 1/10
74/74 [==============================] - 83s 992ms/step - loss: 2.5608 - accuracy: 0.1611 - val_loss: 3.1127 - val_accuracy: 0.1330
Epoch 2/10
74/74 [==============================] - 70s 956ms/step - loss: 2.0874 - accuracy: 0.2812 - val_loss: 2.2926 - val_accuracy: 0.1730
Epoch 3/10
74/74 [==============================] - 70s 947ms/step - loss: 1.9754 - accuracy: 0.3177 - val_loss: 2.0121 - val_accuracy: 0.3065
Epoch 4/10
74/74 [==============================] - 70s 948ms/step - loss: 1.7269 - accuracy: 0.4129 - val_loss: 2.3579 - val_accuracy: 0.2364
Epoch 5/10
74/74 [==============================] - 70s 944ms/step - loss: 1.6156 - accuracy: 0.4497 - val_loss: 2.0647 - val_accuracy: 0.2820
Epoch 6/10
74/74 [==============================] - 70s 954ms/step - loss: 1.4279 - accuracy: 0.5210 - val_loss: 2.1108 - val_accuracy: 0.3389
Epoch 7/10
74/74 [==============================] - 69s 937ms/step - loss: 1.1488 - accuracy: 0.6175 - val_loss: 1.7536 - val_accuracy: 0.4316

Using Transfer Learning (VGG16) to improve accuracy

In [ ]:
from keras.applications import VGG16

# include top should be False to remove the softmax layer
pretrained_model = VGG16(include_top=False, weights='imagenet')
pretrained_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                              

In [ ]:
from keras.utils import to_categorical
# extract train and val features
vgg_features_train = pretrained_model.predict(train)
vgg_features_val = pretrained_model.predict(val)

31/31 [==============================] - 26s 834ms/step


In [ ]:
# OHE target column
train_target = to_categorical(train.labels)
val_target = to_categorical(val.labels)

In [ ]:
model2 = Sequential()
model2.add(Flatten(input_shape=(7,7,512)))
model2.add(Dense(100, activation='relu'))
model2.add(Dropout(0.5))
model2.add(BatchNormalization())
model2.add(Dense(10, activation='softmax'))

# compile the model
model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
model2.fit(vgg_features_train, train_target, epochs=50, batch_size=128, validation_data=(vgg_features_val, val_target))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_3 (Dense)             (None, 100)               2508900   
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 batch_normalization_2 (Batc  (None, 100)              400       
 hNormalization)                                                 
                                                                 
 dense_4 (Dense)             (None, 10)                1010      
                                                                 
Total params: 2,510,310
Trainable params: 2,510,110
Non-trainable params: 200
__________________________________________